In [ ]:
!pip install torch torchvision numpy matplotlib ot


ERROR: Could not find a version that satisfies the requirement ot (from versions: none)
ERROR: No matching distribution found for ot


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

batch_size = 128
train_loader_c10 = torch.utils.data.DataLoader(train_cifar10, batch_size=batch_size, shuffle=True)
test_loader_c10 = torch.utils.data.DataLoader(test_cifar10, batch_size=batch_size, shuffle=False)

train_loader_c100 = torch.utils.data.DataLoader(train_cifar100, batch_size=batch_size, shuffle=True)
test_loader_c100 = torch.utils.data.DataLoader(test_cifar100, batch_size=batch_size, shuffle=False)

print("Datasets loaded successfully.")


100%|██████████| 170M/170M [00:02<00:00, 60.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


100%|██████████| 169M/169M [00:02<00:00, 62.9MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Datasets loaded successfully.


In [ ]:
import torch.nn as nn
import torchvision.models as models

def get_vgg11_model(num_classes):
    model = models.vgg11(pretrained=False)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def get_resnet20_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(512, num_classes)
    return model

print("Models defined successfully.")


Models defined successfully.


In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torch.cuda.amp import GradScaler, autocast

# Step 1: Load and Prepare the Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 and CIFAR-100 datasets
train_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_cifar100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Reduced datasets for faster training
small_train_c10 = Subset(train_cifar10, random.sample(range(len(train_cifar10)), 1000))
small_train_c100 = Subset(train_cifar100, random.sample(range(len(train_cifar100)), 1000))

batch_size = 128
train_loader_c10 = torch.utils.data.DataLoader(small_train_c10, batch_size=batch_size, shuffle=True)
train_loader_c100 = torch.utils.data.DataLoader(small_train_c100, batch_size=batch_size, shuffle=True)

test_loader_c10 = torch.utils.data.DataLoader(test_cifar10, batch_size=batch_size, shuffle=False)
test_loader_c100 = torch.utils.data.DataLoader(test_cifar100, batch_size=batch_size, shuffle=False)

print("Datasets loaded successfully.")

# Step 2: Define Simplified VGG11 Model
class SimpleVGG(nn.Module):
    def __init__(self, num_classes):
        super(SimpleVGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Dynamically calculate the input size for the fully connected layer
        self.flattened_size = self._get_flattened_size()
        self.classifier = nn.Sequential(
            nn.Linear(self.flattened_size, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def _get_flattened_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 3, 32, 32)
            x = self.features(x)
            return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

def get_simple_vgg11_model(num_classes):
    return SimpleVGG(num_classes)

print("Simplified model architecture defined.")

# Step 3: Define Mixed Precision Training Function
def train_model_fast(model, train_loader, epochs=1, max_batches=5, lr=0.01):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scaler = GradScaler()

    for epoch in range(epochs):
        model.train()
        batch_count = 0
        for images, labels in train_loader:
            if batch_count >= max_batches:
                break
            images, labels = images.to(device), labels.to(device)

            with autocast():  # Corrected: Removed device_type argument
                outputs = model(images)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            batch_count += 1
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')
    return model

# Step 4: Train Simplified Models
print("Training models on smaller datasets with simplified architecture...")
vgg11_c10_model1 = train_model_fast(get_simple_vgg11_model(10), train_loader_c10, epochs=1, max_batches=5)
vgg11_c10_model2 = train_model_fast(get_simple_vgg11_model(10), train_loader_c10, epochs=1, max_batches=5)

vgg11_c100_model1 = train_model_fast(get_simple_vgg11_model(100), train_loader_c100, epochs=1, max_batches=5)
vgg11_c100_model2 = train_model_fast(get_simple_vgg11_model(100), train_loader_c100, epochs=1, max_batches=5)

print("Training completed.")

# Step 5: Evaluate the Models
def evaluate_model(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

print("Evaluating models...")
acc_c10_model1 = evaluate_model(vgg11_c10_model1, test_loader_c10)
acc_c10_model2 = evaluate_model(vgg11_c10_model2, test_loader_c10)

acc_c100_model1 = evaluate_model(vgg11_c100_model1, test_loader_c100)
acc_c100_model2 = evaluate_model(vgg11_c100_model2, test_loader_c100)

print(f"Accuracy of CIFAR-10 Model 1: {acc_c10_model1:.2f}%")
print(f"Accuracy of CIFAR-10 Model 2: {acc_c10_model2:.2f}%")
print(f"Accuracy of CIFAR-100 Model 1: {acc_c100_model1:.2f}%")
print(f"Accuracy of CIFAR-100 Model 2: {acc_c100_model2:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Datasets loaded successfully.
Simplified model architecture defined.
Training models on smaller datasets with simplified architecture...


<ipython-input-8-2b5552708b44>:84: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-8-2b5552708b44>:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Corrected: Removed device_type argument


Epoch [1/1], Loss: 2.3019
Epoch [1/1], Loss: 2.3039
Epoch [1/1], Loss: 4.6043
Epoch [1/1], Loss: 4.6050
Training completed.
Evaluating models...
Accuracy of CIFAR-10 Model 1: 9.77%
Accuracy of CIFAR-10 Model 2: 10.70%
Accuracy of CIFAR-100 Model 1: 0.98%
Accuracy of CIFAR-100 Model 2: 0.88%


In [ ]:
!pip install pot


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 11.7 MB/s eta 0:00:00


In [ ]:
from scipy.optimize import linear_sum_assignment
import numpy as np
from sklearn.cross_decomposition import CCA
import torch

# Direct Averaging
def direct_averaging(model_a, model_b):
    averaged_model = {k: (model_a.state_dict()[k] + model_b.state_dict()[k]) / 2
                      for k in model_a.state_dict()}
    model_a.load_state_dict(averaged_model)
    return model_a

# Permute
def permute_weights(model_a, model_b):
    """
    Aligns the weights of model_b to model_a using permutation matching.
    """
    # Extract the first weight tensor from each model
    weight_a = list(model_a.parameters())[0].detach().cpu().numpy()
    weight_b = list(model_b.parameters())[0].detach().cpu().numpy()

    # Ensure the shapes match
    if weight_a.shape != weight_b.shape:
        raise ValueError(f"Shape mismatch between model_a and model_b weights: {weight_a.shape} vs {weight_b.shape}")

    # Handle different tensor dimensions
    if weight_b.ndim == 4:  # Convolutional layer weights: [out_channels, in_channels, kernel_height, kernel_width]
        out_channels, in_channels, kernel_height, kernel_width = weight_a.shape

        # Flatten weights along the in_channels axis
        reshaped_a = weight_a.transpose(1, 0, 2, 3).reshape(in_channels, -1)
        reshaped_b = weight_b.transpose(1, 0, 2, 3).reshape(in_channels, -1)

        # Compute correlation for in_channels only
        correlation_matrix = np.corrcoef(reshaped_a, reshaped_b, rowvar=True)
        correlation_matrix = correlation_matrix[:in_channels, in_channels:]

        # Solve the assignment problem
        row_ind, col_ind = linear_sum_assignment(-correlation_matrix)

        # Apply the permutation to the in_channels of weight_b
        permuted_weight_b = weight_b[:, col_ind, :, :]
    elif weight_b.ndim == 2:  # Fully connected layer weights: [out_features, in_features]
        correlation_matrix = np.corrcoef(weight_a.T, weight_b.T)
        row_ind, col_ind = linear_sum_assignment(-correlation_matrix)
        permuted_weight_b = weight_b[:, col_ind]
    else:
        raise ValueError(f"Unsupported weight dimension: {weight_b.ndim}")

    # Convert permuted weights back to PyTorch tensor
    permuted_weight_b = torch.tensor(permuted_weight_b, dtype=torch.float32)

    # Load the permuted weights into model_b
    model_b_state = model_b.state_dict()
    first_key = list(model_b_state.keys())[0]  # Assume first parameter key corresponds to the weight tensor
    model_b_state[first_key] = permuted_weight_b

    # Update model_b's state_dict
    model_b.load_state_dict(model_b_state, strict=False)

    return model_b




# OT Fusion
def ot_fusion(model_a, model_b):
    """
    Aligns the weights of model_b to model_a using Optimal Transport Fusion.
    """
    # Extract the first weight tensor from each model
    weight_a = list(model_a.parameters())[0].detach().cpu().numpy()
    weight_b = list(model_b.parameters())[0].detach().cpu().numpy()

    # Ensure the shapes match
    if weight_a.shape != weight_b.shape:
        raise ValueError(f"Shape mismatch between model_a and model_b weights: {weight_a.shape} vs {weight_b.shape}")

    # Flatten the weights
    weight_a_flat = weight_a.flatten()
    weight_b_flat = weight_b.flatten()

    # Create a cost matrix as the absolute difference between weights
    cost_matrix = np.abs(weight_a_flat[:, None] - weight_b_flat[None, :])

    # Solve optimal transport using SciPy's linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # Create the transport matrix (binary in this simplified case)
    transport_matrix = np.zeros_like(cost_matrix)
    transport_matrix[row_ind, col_ind] = 1

    # Merge weights using the transport matrix
    aligned_weights = (transport_matrix @ weight_b_flat) + ((1 - transport_matrix) @ weight_a_flat)

    # Reshape aligned weights back to the original shape
    aligned_weights = aligned_weights.reshape(weight_a.shape)

    # Convert aligned weights to PyTorch tensor
    aligned_weights = torch.tensor(aligned_weights, dtype=torch.float32)

    # Update model_a's state_dict with the fused weights
    model_a_state = model_a.state_dict()
    first_key = list(model_a_state.keys())[0]  # Assume first parameter key corresponds to the weight tensor
    model_a_state[first_key] = aligned_weights

    # Load updated state_dict into model_a
    model_a.load_state_dict(model_a_state, strict=False)

    return model_a


# ZipIt
def zipit_merge(model_a, model_b):
    """
    Merges the weights of model_a and model_b using the ZipIt! method.
    """
    # Extract the state dictionaries from both models
    state_dict_a = model_a.state_dict()
    state_dict_b = model_b.state_dict()

    # Initialize a new state_dict for the merged model
    merged_state_dict = {}

    # Iterate through all keys in the state_dict
    for key in state_dict_a:
        if key in state_dict_b:
            # Perform element-wise maximum for matching keys
            merged_state_dict[key] = torch.max(state_dict_a[key], state_dict_b[key])
        else:
            # If key is missing in model_b, keep model_a's weights
            merged_state_dict[key] = state_dict_a[key]

    # Load the merged state_dict into model_a
    model_a.load_state_dict(merged_state_dict)

    return model_a


# CCA Merge
def cca_merge(model_a, model_b):
    weight_a = list(model_a.parameters())[0].detach().cpu().numpy()
    weight_b = list(model_b.parameters())[0].detach().cpu().numpy()

    cca = CCA(n_components=min(weight_a.shape[1], weight_b.shape[1]))
    cca.fit(weight_a, weight_b)
    aligned_a, aligned_b = cca.transform(weight_a, weight_b)
    aligned_weights = 0.5 * (aligned_a + aligned_b)
    model_a.load_state_dict({'weight': aligned_weights})
    return model_a

print("All fusion methods implemented successfully.")


All fusion methods implemented successfully.


In [ ]:
def evaluate_model(model, test_loader):
    """
    Evaluate the model on the test dataset.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

methods = ['Direct Averaging', 'Permute', 'OT Fusion', 'ZipIt', 'CCA Merge']
results = []

# Apply all fusion methods
for method in methods:
    if method == 'Direct Averaging':
        merged_model = direct_averaging(vgg11_c10_model1, vgg11_c10_model2)
    elif method == 'Permute':
        merged_model = permute_weights(vgg11_c10_model1, vgg11_c10_model2)
    elif method == 'OT Fusion':
        merged_model = ot_fusion(vgg11_c10_model1, vgg11_c10_model2)
    elif method == 'ZipIt':
        merged_model = zipit_merge(vgg11_c10_model1, vgg11_c10_model2)
    elif method == 'CCA Merge':
        merged_model = cca_merge(vgg11_c10_model1, vgg11_c10_model2)

    # Evaluate the merged model
    acc = evaluate_model(merged_model, test_loader_c10)
    results.append((method, acc))
    print(f"{method} Accuracy: {acc:.2f}%")


Direct Averaging Accuracy: 11.47%
Permute Accuracy: 10.70%
OT Fusion Accuracy: 12.09%
ZipIt Accuracy: 12.11%


ValueError: Found array with dim 4. CCA expected <= 2.

In [ ]:
import pandas as pd

# Create results table
df = pd.DataFrame(results, columns=["Method", "Accuracy (%)"])
print(df)

# Display results in a bar plot
import matplotlib.pyplot as plt

plt.bar(df["Method"], df["Accuracy (%)"])
plt.xlabel('Fusion Methods')
plt.ylabel('Accuracy (%)')
plt.title('Fusion Methods Comparison on CIFAR-10')
plt.xticks(rotation=45)
plt.show()
